In [1]:
from pathlib import Path
import torch

cmip_path = Path("/data0/datasets/weatherbench/data/weatherbench/cmip6/5.625")   # replace with path
era_path = Path("/data0/datasets/weatherbench/data/weatherbench/era5/5.625deg")

In [2]:
import xarray as xr

import sys
sys.path.insert(0, '/home/seongbin/climate-learn/src')

In [3]:
from climate_learn.utils.datetime import Year, Days, Hours
from climate_learn.data import DataModule

/home/seongbin/climate-learn/src/climate_learn/data/modules/cmip6_module.py:77: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if var is "geopotential":
/home/seongbin/climate-learn/src/climate_learn/data/modules/cmip6_module.py:84: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if var is "geopotential":


In [5]:
era5_data_module = DataModule(
    dataset = "ERA5",
    task = "forecasting",
    root_dir = era_path,
    in_vars = ["geopotential"],
    out_vars = ["geopotential_500"],
    train_start_year = Year(1979),
    val_start_year = Year(2011),
    test_start_year = Year(2013),
    end_year = Year(2014),
    pred_range = Days(5),
    subsample = Hours(6),
    batch_size = 128,
    num_workers = 16
) 

Creating train dataset
update


  0%|          | 0/32 [00:00<?, ?it/s]

out 280512
in 280512
Finished inp and out _data
True
Almost done!
Creating val dataset
update


  0%|          | 0/2 [00:00<?, ?it/s]

out 17544
in 17544
Finished inp and out _data
True
Almost done!
Creating test dataset
update


  0%|          | 0/2 [00:00<?, ?it/s]

out 17520
in 17520
Finished inp and out _data
True
Almost done!


In [4]:
cmip6_data_module = DataModule(
    dataset = "CMIP6",
    task = "forecasting",
    root_dir = cmip_path,
    in_vars = ["geopotential"],
    out_vars = ["geopotential_500"],
    train_start_year = Year(1850),
    val_start_year = Year(2011),
    test_start_year = Year(2013),
    end_year = Year(2014),
    pred_range = Days(5),
    subsample = Hours(6),
    batch_size = 128,
    num_workers = 16
) 

Creating train dataset


  0%|          | 0/161 [00:00<?, ?it/s]

in 235039
out 235039
Creating val dataset


  0%|          | 0/2 [00:00<?, ?it/s]

in 2918
out 2918
Creating test dataset


  0%|          | 0/2 [00:00<?, ?it/s]

in 2918
out 2918


In [16]:
cmip6_data_module.train_dataset.out_transform

Normalize(mean=[530730.4], std=[32671.979])

In [19]:
from climate_learn.models import load_model


cmip_model_kwargs = {
    "in_channels": len(cmip6_data_module.hparams.in_vars),
    "out_channels": len(cmip6_data_module.hparams.out_vars),
    "n_blocks": 4
}

optim_kwargs = {
    "lr": 1e-4,
    "weight_decay": 1e-5,
    "warmup_epochs": 1,
    "max_epochs": 5,
}

cmip_model_module = load_model(name = "resnet", task = "forecasting", model_kwargs = cmip_model_kwargs, optim_kwargs = optim_kwargs)

In [20]:
from climate_learn.models import set_climatology
set_climatology(cmip_model_module, cmip6_data_module)

In [21]:
from climate_learn.training import Trainer, WandbLogger

cmip_trainer = Trainer(
    seed = 0,
    accelerator = "gpu",
    precision = 16,
    max_epochs = 5,
    # logger = WandbLogger(project = "climate_tutorial", name = "forecast-vit")
)

In [22]:
cmip_trainer.fit(cmip_model_module, cmip6_data_module)

/home/seongbin/miniconda3/envs/climate_pretrain/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning: Checkpoint directory /data0/ckpts/seongbin/cross-train-z500/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃    ┃ Name                       ┃ Type              ┃ Params ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0  │ net                        │ ResNet            │  1.2 M │
│ 1  │ net.activation             │ LeakyReLU         │      0 │
│ 2  │ net.image_proj             │ PeriodicConv2D    │  6.4 K │
│ 3  │ net.image_proj.padding     │ PeriodicPadding2D │      0 │
│ 4  │ net.image_proj.conv        │ Conv2d            │  6.4 K │
│ 5  │ net.blocks                 │ ModuleList        │  1.2 M │
│ 6  │ net.blocks.0               │ ResidualBlock     │  295 K │
│ 7  │ net.blocks.0.activation    │ LeakyReLU         │      0 │
│ 8  │ net.blocks.0.conv1         │ PeriodicConv2D    │  147 K │
│ 9  │ net.blocks.0.conv1.padding │ PeriodicPadding2D │      0 │
│ 10 │ net.blocks.0.conv1.conv    │ Conv2d            │  147 K │
│ 11 │ net.blocks.0.conv2         │ PeriodicConv2D    │  147 K │
│ 12 │ net.blocks.0.conv2.padding │ PeriodicPadding2D │      0 │
│ 13 │ net.blocks.0.conv2.conv    │ Conv2d            │  147 K │
│ 14 │ net.blocks.0.shortcut      │ Identity          │      0 │
│ 15 │ net.blocks.0.norm1         │ BatchNorm2d       │    256 │
│ 16 │ net.blocks.0.norm2         │ BatchNorm2d       │    256 │
│ 17 │ net.blocks.0.drop          │ Dropout           │      0 │
│ 18 │ net.blocks.1               │ ResidualBlock     │  295 K │
│ 19 │ net.blocks.1.activation    │ LeakyReLU         │      0 │
│ 20 │ net.blocks.1.conv1         │ PeriodicConv2D    │  147 K │
│ 21 │ net.blocks.1.conv1.padding │ PeriodicPadding2D │      0 │
│ 22 │ net.blocks.1.conv1.conv    │ Conv2d            │  147 K │
│ 23 │ net.blocks.1.conv2         │ PeriodicConv2D    │  147 K │
│ 24 │ net.blocks.1.conv2.padding │ PeriodicPadding2D │      0 │
│ 25 │ net.blocks.1.conv2.conv    │ Conv2d            │  147 K │
│ 26 │ net.blocks.1.shortcut      │ Identity          │      0 │
│ 27 │ net.blocks.1.norm1         │ BatchNorm2d       │    256 │
│ 28 │ net.blocks.1.norm2         │ BatchNorm2d       │    256 │
│ 29 │ net.blocks.1.drop          │ Dropout           │      0 │
│ 30 │ net.blocks.2               │ ResidualBlock     │  295 K │
│ 31 │ net.blocks.2.activation    │ LeakyReLU         │      0 │
│ 32 │ net.blocks.2.conv1         │ PeriodicConv2D    │  147 K │
│ 33 │ net.blocks.2.conv1.padding │ PeriodicPadding2D │      0 │
│ 34 │ net.blocks.2.conv1.conv    │ Conv2d            │  147 K │
│ 35 │ net.blocks.2.conv2         │ PeriodicConv2D    │  147 K │
│ 36 │ net.blocks.2.conv2.padding │ PeriodicPadding2D │      0 │
│ 37 │ net.blocks.2.conv2.conv    │ Conv2d            │  147 K │
│ 38 │ net.blocks.2.shortcut      │ Identity          │      0 │
│ 39 │ net.blocks.2.norm1         │ BatchNorm2d       │    256 │
│ 40 │ net.blocks.2.norm2         │ BatchNorm2d       │    256 │
│ 41 │ net.blocks.2.drop          │ Dropout           │      0 │
│ 42 │ net.blocks.3               │ ResidualBlock     │  295 K │
│ 43 │ net.blocks.3.activation    │ LeakyReLU         │      0 │
│ 44 │ net.blocks.3.conv1         │ PeriodicConv2D    │  147 K │
│ 45 │ net.blocks.3.conv1.padding │ PeriodicPadding2D │      0 │
│ 46 │ net.blocks.3.conv1.conv    │ Conv2d            │  147 K │
│ 47 │ net.blocks.3.conv2         │ PeriodicConv2D    │  147 K │
│ 48 │ net.blocks.3.conv2.padding │ PeriodicPadding2D │      0 │
│ 49 │ net.blocks.3.conv2.conv    │ Conv2d            │  147 K │
│ 50 │ net.blocks.3.shortcut      │ Identity          │      0 │
│ 51 │ net.blocks.3.norm1         │ BatchNorm2d       │    256 │
│ 52 │ net.blocks.3.norm2         │ BatchNorm2d       │    256 │
│ 53 │ net.blocks.3.drop          │ Dropout           │      0 │
│ 54 │ net.norm                   │ BatchNorm2d       │    256 │
│ 55 │ net.final                  │ PeriodicConv2D    │  6.3 K │
│ 56 │ net.final.padding          │ PeriodicPadding2D │      0 │
│ 57 │ net.final.conv             │ Conv2d            │  6.3 K │
│ 58 │ denormalization            │

Trainable params: 1.2 M                                                                      
Non-trainable params: 0                                                                      
Total params: 1.2 M                                                                          
Total estimated model params size (MB): 2

Output()

In [10]:
era5_model_kwargs = {
    "in_channels": len(era5_data_module.hparams.in_vars),
    "out_channels": len(era5_data_module.hparams.out_vars),
    "n_blocks": 4
}

optim_kwargs = {
    "lr": 1e-4,
    "weight_decay": 1e-5,
    "warmup_epochs": 1,
    "max_epochs": 5,
}

era5_model_module = load_model(name = "resnet", task = "forecasting", model_kwargs = era5_model_kwargs, optim_kwargs = optim_kwargs)

In [11]:
set_climatology(era5_model_module, era5_data_module)

In [12]:
era5_trainer = Trainer(
    seed = 0,
    accelerator = "gpu",
    precision = 16,
    max_epochs = 5,
    # logger = WandbLogger(project = "climate_tutorial", name = "forecast-vit")
)

In [13]:
era5_trainer.fit(era5_model_module, era5_data_module)

/home/seongbin/miniconda3/envs/climate_pretrain/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning: Checkpoint directory /data0/ckpts/seongbin/cross-train-z500/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃    ┃ Name                       ┃ Type              ┃ Params ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0  │ net                        │ ResNet            │  1.2 M │
│ 1  │ net.activation             │ LeakyReLU         │      0 │
│ 2  │ net.image_proj             │ PeriodicConv2D    │  6.4 K │
│ 3  │ net.image_proj.padding     │ PeriodicPadding2D │      0 │
│ 4  │ net.image_proj.conv        │ Conv2d            │  6.4 K │
│ 5  │ net.blocks                 │ ModuleList        │  1.2 M │
│ 6  │ net.blocks.0               │ ResidualBlock     │  295 K │
│ 7  │ net.blocks.0.activation    │ LeakyReLU         │      0 │
│ 8  │ net.blocks.0.conv1         │ PeriodicConv2D    │  147 K │
│ 9  │ net.blocks.0.conv1.padding │ PeriodicPadding2D │      0 │
│ 10 │ net.blocks.0.conv1.conv    │ Conv2d            │  147 K │
│ 11 │ net.blocks.0.conv2         │ PeriodicConv2D    │  147 K │
│ 12 │ net.blocks.0.conv2.padding │ PeriodicPadding2D │      0 │
│ 13 │ net.blocks.0.conv2.conv    │ Conv2d            │  147 K │
│ 14 │ net.blocks.0.shortcut      │ Identity          │      0 │
│ 15 │ net.blocks.0.norm1         │ BatchNorm2d       │    256 │
│ 16 │ net.blocks.0.norm2         │ BatchNorm2d       │    256 │
│ 17 │ net.blocks.0.drop          │ Dropout           │      0 │
│ 18 │ net.blocks.1               │ ResidualBlock     │  295 K │
│ 19 │ net.blocks.1.activation    │ LeakyReLU         │      0 │
│ 20 │ net.blocks.1.conv1         │ PeriodicConv2D    │  147 K │
│ 21 │ net.blocks.1.conv1.padding │ PeriodicPadding2D │      0 │
│ 22 │ net.blocks.1.conv1.conv    │ Conv2d            │  147 K │
│ 23 │ net.blocks.1.conv2         │ PeriodicConv2D    │  147 K │
│ 24 │ net.blocks.1.conv2.padding │ PeriodicPadding2D │      0 │
│ 25 │ net.blocks.1.conv2.conv    │ Conv2d            │  147 K │
│ 26 │ net.blocks.1.shortcut      │ Identity          │      0 │
│ 27 │ net.blocks.1.norm1         │ BatchNorm2d       │    256 │
│ 28 │ net.blocks.1.norm2         │ BatchNorm2d       │    256 │
│ 29 │ net.blocks.1.drop          │ Dropout           │      0 │
│ 30 │ net.blocks.2               │ ResidualBlock     │  295 K │
│ 31 │ net.blocks.2.activation    │ LeakyReLU         │      0 │
│ 32 │ net.blocks.2.conv1         │ PeriodicConv2D    │  147 K │
│ 33 │ net.blocks.2.conv1.padding │ PeriodicPadding2D │      0 │
│ 34 │ net.blocks.2.conv1.conv    │ Conv2d            │  147 K │
│ 35 │ net.blocks.2.conv2         │ PeriodicConv2D    │  147 K │
│ 36 │ net.blocks.2.conv2.padding │ PeriodicPadding2D │      0 │
│ 37 │ net.blocks.2.conv2.conv    │ Conv2d            │  147 K │
│ 38 │ net.blocks.2.shortcut      │ Identity          │      0 │
│ 39 │ net.blocks.2.norm1         │ BatchNorm2d       │    256 │
│ 40 │ net.blocks.2.norm2         │ BatchNorm2d       │    256 │
│ 41 │ net.blocks.2.drop          │ Dropout           │      0 │
│ 42 │ net.blocks.3               │ ResidualBlock     │  295 K │
│ 43 │ net.blocks.3.activation    │ LeakyReLU         │      0 │
│ 44 │ net.blocks.3.conv1         │ PeriodicConv2D    │  147 K │
│ 45 │ net.blocks.3.conv1.padding │ PeriodicPadding2D │      0 │
│ 46 │ net.blocks.3.conv1.conv    │ Conv2d            │  147 K │
│ 47 │ net.blocks.3.conv2         │ PeriodicConv2D    │  147 K │
│ 48 │ net.blocks.3.conv2.padding │ PeriodicPadding2D │      0 │
│ 49 │ net.blocks.3.conv2.conv    │ Conv2d            │  147 K │
│ 50 │ net.blocks.3.shortcut      │ Identity          │      0 │
│ 51 │ net.blocks.3.norm1         │ BatchNorm2d       │    256 │
│ 52 │ net.blocks.3.norm2         │ BatchNorm2d       │    256 │
│ 53 │ net.blocks.3.drop          │ Dropout           │      0 │
│ 54 │ net.norm                   │ BatchNorm2d       │    256 │
│ 55 │ net.final                  │ PeriodicConv2D    │  6.3 K │
│ 56 │ net.final.padding          │ PeriodicPadding2D │      0 │
│ 57 │ net.final.conv             │ Conv2d            │  6.3 K │
│ 58 │ denormalization            │

Trainable params: 1.2 M                                                                      
Non-trainable params: 0                                                                      
Total params: 1.2 M                                                                          
Total estimated model params size (MB): 2

Output()

In [23]:
from climate_learn.models import fit_lin_reg_baseline
fit_lin_reg_baseline(cmip_model_module, cmip6_data_module, reg_hparam=0.0)

In [20]:
fit_lin_reg_baseline(era5_model_module, era5_data_module, reg_hparam=0.0)

# Data

Days: 5, Var: geopotential 500, model: resnet

train_start_year = 1979 / 1850, val_start_year = 2011, test_start_year = 2013, end_year = 2014

In [24]:
# cmip -> era
cmip_trainer.test(cmip_model_module, era5_data_module)

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                 Test metric                 ┃                DataLoader 0                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/acc_geopotential_500_day_5       │                     nan                     │
│     test/w_rmse_geopotential_500_day_5      │                     inf                     │
│ test_climatology_baseline/w_rmse_geopotent… │             10525.370441225194              │
│ test_persistence_baseline/w_rmse_geopotent… │              7327.535915257272              │
│ test_ridge_regression_baseline/w_rmse_geop… │              35301.89840019362              │
└─────────────────────────────────────────────┴─────────────────────────────────────────────┘

In [25]:
# cmip -> cmip
cmip_trainer.test(cmip_model_module, cmip6_data_module)

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                 Test metric                 ┃                DataLoader 0                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/acc_geopotential_500_day_5       │                     nan                     │
│     test/w_rmse_geopotential_500_day_5      │                     inf                     │
│ test_climatology_baseline/w_rmse_geopotent… │             11486.937617791506              │
│ test_persistence_baseline/w_rmse_geopotent… │             11165.575415549352              │
│ test_ridge_regression_baseline/w_rmse_geop… │              36178.12742491186              │
└─────────────────────────────────────────────┴─────────────────────────────────────────────┘

In [23]:
# era -> era
era5_trainer.test(era5_model_module, era5_data_module)

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                 Test metric                 ┃                DataLoader 0                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/acc_geopotential_500_day_5       │             0.6340033525213865              │
│     test/w_rmse_geopotential_500_day_5      │              772.6244066145214              │
│ test_climatology_baseline/w_rmse_geopotent… │              1068.250577807763              │
│ test_persistence_baseline/w_rmse_geopotent… │              751.5648628721129              │
│ test_ridge_regression_baseline/w_rmse_geop… │              809.3986487426973              │
└─────────────────────────────────────────────┴─────────────────────────────────────────────┘

In [24]:
# era -> cmip
era5_trainer.test(era5_model_module, cmip6_data_module)

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                 Test metric                 ┃                DataLoader 0                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/acc_geopotential_500_day_5       │             0.4299115046717596              │
│     test/w_rmse_geopotential_500_day_5      │             1066.8455588532515              │
│ test_climatology_baseline/w_rmse_geopotent… │             1191.0452447533505              │
│ test_persistence_baseline/w_rmse_geopotent… │             1145.2218501535338              │
│ test_ridge_regression_baseline/w_rmse_geop… │             1181.3169769302337              │
└─────────────────────────────────────────────┴─────────────────────────────────────────────┘

## Unet??

In [ ]:
cmip_unet_model_module = load_model(name = "unet", task = "forecasting", model_kwargs = cmip_model_kwargs, optim_kwargs = optim_kwargs)
era5_unet_model_module = load_model(name = "unet", task = "forecasting", model_kwargs = era5_model_kwargs, optim_kwargs = optim_kwargs)

In [ ]:
set_climatology(era5_unet_model_module, era5_data_module)
set_climatology(cmip_unet_model_module, cmip6_data_module)

In [ ]:
cmip_unet_trainer = Trainer(
    seed = 0,
    accelerator = "gpu",
    precision = 16,
    max_epochs = 5,
    # logger = WandbLogger(project = "climate_tutorial", name = "forecast-vit")
)
cmip_unet_trainer.fit(cmip_unet_model_module, cmip6_data_module)

In [ ]:
era5_unet_trainer = Trainer(
    seed = 0,
    accelerator = "gpu",
    precision = 16,
    max_epochs = 5,
    # logger = WandbLogger(project = "climate_tutorial", name = "forecast-vit")
)
era5_unet_trainer.fit(era5_unet_model_module, era5_data_module)

In [ ]:
# cmip -> era
cmip_unet_trainer.test(cmip_unet_model_module, era5_data_module)

In [ ]:
# cmip -> cmip
cmip_unet_trainer.test(cmip_unet_model_module, cmip6_data_module)

In [ ]:
# era -> era
era5_unet_trainer.test(era5_unet_model_module, era5_data_module)

In [ ]:
# era -> cmip
era5_unet_trainer.test(era5_unet_model_module, cmip6_data_module)

## ViT???

In [ ]:
cmip_vit_model_kwargs = {
    "img_size": [32, 64],
    "patch_size": 2,
    "drop_path": 0.1,
    "drop_rate": 0.1,
    "learn_pos_emb": True,
    "in_vars": cmip6_data_module.hparams.in_vars,
    "out_vars": cmip6_data_module.hparams.out_vars,
    "embed_dim": 128,
    "depth": 8,
    "decoder_depth": 0,
    "num_heads": 4,
    "mlp_ratio": 4,
}

era5_vit_model_kwargs = {
    "img_size": [32, 64],
    "patch_size": 2,
    "drop_path": 0.1,
    "drop_rate": 0.1,
    "learn_pos_emb": True,
    "in_vars": era5_model_kwargs.hparams.in_vars,
    "out_vars": era5_model_kwargs.hparams.out_vars,
    "embed_dim": 128,
    "depth": 8,
    "decoder_depth": 0,
    "num_heads": 4,
    "mlp_ratio": 4,
}

# model_kwargs = {
#     "in_vars": ["2m_temperature"],
#     "out_vars": ["2m_temperature"],
#     "img_size": [32, 64]
# }

In [ ]:
cmip_vit_model_module = load_model(name = "vit", task = "forecasting", model_kwargs = cmip_vit_model_kwargs, optim_kwargs = optim_kwargs)
era5_vit_model_module = load_model(name = "vit", task = "forecasting", model_kwargs = era5_vit_model_kwargs, optim_kwargs = optim_kwargs)

In [ ]:
set_climatology(era5_vit_model_module, era5_data_module)
set_climatology(cmip_vit_model_module, cmip6_data_module)

In [ ]:
cmip_vit_trainer = Trainer(
    seed = 0,
    accelerator = "gpu",
    precision = 16,
    max_epochs = 5,
    # logger = WandbLogger(project = "climate_tutorial", name = "forecast-vit")
)
cmip_vit_trainer.fit(cmip_vit_model_module, cmip6_data_module)

In [ ]:
era5_vit_trainer = Trainer(
    seed = 0,
    accelerator = "gpu",
    precision = 16,
    max_epochs = 5,
    # logger = WandbLogger(project = "climate_tutorial", name = "forecast-vit")
)
era5_vit_trainer.fit(era5_vit_model_module, era5_data_module)

In [ ]:
# cmip -> era
cmip_vit_trainer.test(cmip_vit_model_module, era5_data_module)

In [ ]:
# cmip -> cmip
cmip_vit_trainer.test(cmip_vit_model_module, cmip6_data_module)

In [ ]:
# era -> era
era5_vit_trainer.test(era5_vit_model_module, era5_data_module)

In [ ]:
# era -> cmip
era5_vit_trainer.test(era5_vit_model_module, cmip6_data_module)